In [3]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import random

# Projet: Modèles linéaires:  Adaline et Regression Logistique

Nous allons nous intéresser à l'implémentation d'un algorithme de descente de gradient pour trouver le meilleur paramètre d'un module Adaline ou de regression logistique.

Pout cela, on implémentera un algorithme de descente de gradient stochastique que nous avons vu au TP précédent et dont le pseudo-code peut être résumé comme suit:

```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    choisir un exemple (x,y) de Train de façon aléatoire
    calculer h = w*x
    calculer Loss(h, y)
    w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
output: w
```
où "eta" est le pas de la descente de gradient (exemple: eta=0.01).

Si on veut imprimer l'erreur tous les "m" pas de gradient:
```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    err = 0
    for i in range(m):
        choisir un exemple (x,y) de Train de façon aléatoire
        calculer h = w*x
        err += Loss(h, y)
        w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
    print(err)
output: w
```

Pour un poids $w$, on définit $h_\mathbf{w}(\mathbf{x})=w_0x_0+w_1x_1+...w_dx_d$. Pour chacun des deux modèles, et pour un exemple $(\mathbf{x},y)$, la prédiction $\hat{y}(\mathbf{w}, \mathbf{x})$ et la fonction de coût  $\mathcal{L}(\mathbf{w}, \mathbf{x})$ sont: 
- Adaline: $\hat{y}(\mathbf{w}, \mathbf{x}) = h_\mathbf{w}(x)$ et $$\mathcal{L}(\mathbf{w})=(y-\hat{y}(\mathbf{w},\mathbf{x}))^2=(y-h_\mathbf{w}(\mathbf{x}))^2,$$
- Régression logistique: $\hat{y}(w, x) = 1/(1+e^{-h_{\mathbf{w}}(\mathbf{x})})$ et $$\mathcal{L}(\mathbf{w}, x) = - y \log \hat{y}(\mathbf{w},\mathbf{x}) - (1-y)\log(1-\hat{y}(\mathbf{w},\mathbf{x})) = \log(1+e^{h_{\mathbf{w}}(\mathbf{x})})-yh_\mathbf{w}(\mathbf{x}),$$

Nous avons vu les gradients de ces fonctions en TD.

## Partie 1: implémentation de l'algorithme et exemple du "ET logique"

<font color='red'><b>Question 1:</b> le "ET logique".</font> Créer une liste de 4 éléments où chaque élément est un couple de la forme `[x,y]`, avec `x=[1,x1,x2]` et `y = x1 and x2`. Il y a 4 éléments car `x1` et `x2` peuvent chacun prendre la valeur `0` ou `1` (chacun de ces 4 éléments est une liste dont le premier élément est les attributs de l'exemple et le deuxième élément est la classe de l'exemple).

In [62]:
liste = [0] * 4
for x1 in range(2):
    for x2 in range(2):
        liste[x1*1+x2*2] = [[1,x1,x2], x1 and x2] 

X = np.array([x[0] for x in liste])
Y = np.array([x[1] for x in liste])

X,Y

(array([[1, 0, 0],
        [1, 1, 0],
        [1, 0, 1],
        [1, 1, 1]]),
 array([0, 0, 0, 1]))

<font color='red'><b>Question 2:</b></font> Coder le modèle Adaline et le modèle de régression logistique et le faire tourner sur le modèle de "ET logique". Calculer le taux d'erreur de votre algorithme sur cette base (où une erreur est comptabilisé si la prédiction est plus proche de la fausse classe que de la vraie classe). 


In [87]:
import random as rd

# Adaline
def h(w,x):
    return w@x

def loss_adaline(p,y):
    return (y-p)**2

def adaline_grad(w,x,y):
    return 2*(h(w,x) - y)*x

# Regression logistique
def prediction_regression_logistique(w,x):
    return 1/(1 + np.exp(-h(w,x)))

def loss_regression_logistique(p,y):
    p = np.clip(p, 1e-10, 1-1e-10)
    return -y*np.log(p) - (1-y)*np.log(1-p)

def regression_logistique_grad(w,x,y):
    return (prediction_regression_logistique(w,x)-y)*x

# Gradient
def gradient(grad, w, x, y):
    return grad(w,x,y)

# Learning rate    
def eta_001(t):
    return 0.01

def eta_01(t):
    return 0.1
    
# Stochastic Gradient Descent
def SGD(X, Y, Maxep, eta, p, loss_function, g, err_printing=True):
    w = np.random.rand(len(X[0])) * 0.01
    for epoque in range(Maxep):
        i = rd.randint(0, len(Y) - 1)
        pred = p(w, X[i])
        if err_printing:
            print(loss_function(pred, Y[i]))
        w = w - eta(epoque) * g(w, X[i], Y[i])
    return w

def SGD_with_error_printing(X, Y, Maxep, eta, m, p, loss_function, g, err_printing=True):
    w = np.random.rand(len(X[0])) * 0.01
    for epoque in range(Maxep):
        err = 0
        for _ in range(m):
            i = rd.randint(0, len(Y) - 1)
            pred = p(w, X[i])
            err += loss_function(pred, Y[i])
            w = w - eta(epoque) * g(w, X[i], Y[i])
        if err_printing:
            print("epoque ", epoque, ": ", err)
    return w

# Training Adaline on "ET logique"
w_adaline = SGD(X, Y, 100, eta_001, h, loss_adaline, adaline_grad, False)
print("Final weights for Adaline on 'ET logique':", w_adaline)
# Training Logistic Regression on "ET logique"
w_logistic = SGD_with_error_printing(X, Y, 100, eta_01, 2, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
print("Final weights for Logistic Regression on 'ET logique':", w_logistic)

def calculate_error_rate(w, X, Y, prediction_function, rate_printing=False):
    errors = 0
    for i in range(len(Y)):
        pred = prediction_function(w, X[i])
        if rate_printing:
            print(f"P: {pred}, Y: {Y[i]}")
        if (pred >= 0.5 and Y[i] == 0) or (pred < 0.5 and Y[i] == 1):
            errors += 1
    return errors / len(Y)


# Function to calculate error rate multiple times
def calculate_multiple_error_rates(X, Y, num_iterations, model, prediction_function, eta, Maxep):
    if model == 'adaline':
        print("Training Adaline...") 
    elif model == 'logistic':
        print("Training Logistic Regression...")

    for _ in range(num_iterations):
        if model == 'adaline':
            w = SGD(X, Y, Maxep, eta, h, loss_adaline, adaline_grad, False)
        elif model == 'logistic':
            w = SGD(X, Y, Maxep, eta, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
        error_rate = calculate_error_rate(w, X, Y, prediction_function, False)
        print("Error rate:", error_rate)


# Calculate error rates for Adaline and Logistic Regression
num_iterations = 20

# calculate_multiple_error_rates(X, Y, num_iterations, 'adaline', h, eta_001, 100)
calculate_multiple_error_rates(X, Y, num_iterations, 'adaline', h, eta_01, 100)
# calculate_multiple_error_rates(X, Y, num_iterations, 'logistic', prediction_regression_logistique, eta_01, 100)
# calculate_multiple_error_rates(X, Y, num_iterations, 'logistic', prediction_regression_logistique, eta_01, 200)
calculate_multiple_error_rates(X, Y, num_iterations, 'logistic', prediction_regression_logistique, eta_01, 300)

Final weights for Adaline on 'ET logique': [0.09753723 0.22911396 0.24058637]
Final weights for Logistic Regression on 'ET logique': [-2.23846034  0.81422235  0.69427568]
Training Adaline...
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.25
Error rate: 0.25
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.25
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Training Logistic Regression...
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.25
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0
Error rate: 0.0


## Partie 2: premiers tests avec une base de donnée réelle

<font color='red'><b>Question 3:</b></font> Nous allons maintenant nous intéresser au comportement de ces modèles sur la base SONAR de la collection UCI (http://archive.ics.uci.edu/ml/index.php). Cette base contient 208 exemples en dimension 60 séparés par `,` et la dernière élément correspond à la classe de l'exemple.

    1. Télécharger la collection avec la fonction read_table de la librairie pandas (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html). Les options nécessaires sont `sep=','` et `header=None`  
    2. Créer une liste de listes correspondant à la collection; pour cela initialiser la première liste et en parcourant chaque ligne de la matrice de données; créer une liste associée en remplaçant le dernier élément par `0` ou `+1` et insérer la dans la première liste. 
    Indication: Utiliser la fonction `loc`. 
    3. Écrire une fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable (indication: on pourra utiliser les fonctions `shuffle` de la librairie `random` et `train_test_split` de la librairie `sklearn.model_selection`)
    

In [88]:
df = pd.read_table('Data/Sonar/sonar.all-data', sep = ',', header = None)

x = np.zeros((len(df), 60))
y = np.zeros(208)

for i in range(60):
    x[:,i] = df[i]

for i in range(208):
    if df[60][i] == 'R': y[i] = 1
    elif df[60][i]: y[i] = 0

def generate_train_test_random(x, y):
    data = list(zip(x, y))
    rd.shuffle(data)
    new_X = np.array([i for i, j in data])
    new_Y = np.array([j for i, j in data])
    x_train, x_test, y_train, y_test = train_test_split(new_X, new_Y, test_size=0.25)
    return x_train, x_test, y_train, y_test



<font color='red'><b>Question 4:</b></font> Appliquer ces modèles sur cette base en prenant comme $MaxEp=500$, le pas d'apprentissage $\eta=0.1$ et en choisissant les bases Train et Test de façon aléatoire; Reporter l'erreur moyenne de ces modèles obtenues sur les Tests?  Refaire l'opération 3 fois avec trois randomisations différentes. 


In [89]:
MaxEp = 500

for _ in range(3):
    x_train, x_test, y_train, y_test = generate_train_test_random(x, y)
    out = "iter " + str(_) + ":\n"   
    out += "Training Adaline on sonar data...\n"
    w_adaline = SGD(x_train, y_train, MaxEp, eta_001, h, loss_adaline, adaline_grad, False)
    error_rate = []
    for x_i, y_i in zip(x_test, y_test):
        error_rate.append(loss_adaline(h(w_adaline, x_i), y_i))
    out += "Error rate: " + str(np.mean(error_rate)) + "\n"
    
    out += "Training Logistic Regression on sonar data...\n"
    w_logistic = SGD(x_train, y_train, MaxEp, eta_01, prediction_regression_logistique, loss_regression_logistique, regression_logistique_grad, False)
    error_rate = []
    for x_i, y_i in zip(x_test, y_test):
        error_rate.append(loss_regression_logistique(prediction_regression_logistique(w_logistic, x_i), y_i))
    out += "Error rate: " + str(np.mean(error_rate)) + "\n"
    print(out)

iter 0:
Training Adaline on sonar data...
Error rate: 0.18363731655300558
Training Logistic Regression on sonar data...
Error rate: 0.5713975623046115

iter 1:
Training Adaline on sonar data...
Error rate: 0.19886724528491245
Training Logistic Regression on sonar data...
Error rate: 0.5196932645371463

iter 2:
Training Adaline on sonar data...
Error rate: 0.20051839986274403
Training Logistic Regression on sonar data...
Error rate: 0.5524620074370019



  | Collection | Adaline     | Régression Logistique |
  |------------|-------------|-----------------------|
  |   SONAR (réplica 1)   |             |                       |
  |   SONAR (réplica 2)   |             |                       |
  |   SONAR (réplica 3)   |             |                       |


## Partie 3: normalisation

Nous allons étudier l'impact de la nomralisation sur les prédictions. Pour cela nous considérons deux stratégies de normalisation communément utilisées dans la littérature:
* Stratégie <i>max</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la valeur maximale de cette caractéristiques
* Stratégie <i>norme</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la norme de ce vecteur.

Nous considérons ces trois autres collections de la base UCI:

        * https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29
        * https://archive.ics.uci.edu/ml/datasets/spambase
        * https://archive.ics.uci.edu/ml/datasets/ionosphere

<font color='red'><b>Question 5:</b></font> Ecrire une fonction qui prend en entrée la collection des données et qui retourne la collections normalisée suivant les stratégies <i>max</i> et <i>norme</i>. 
        

In [92]:
BreastCancer = pd.read_table('Data/BreastCancer/wdbc.data', header=None, sep=',')
Ionosphere = pd.read_csv('Data/Ionosphere/ionosphere.data', header=None, sep=',')
Spambase = pd.read_csv('Data/Spambase/spambase.data', header=None, sep=',')

def produce_collection(dataFrame, numL, numC, targetC, classType):
    """
    Entree: dataFrame, numL
    Sortie: features, target
    """
    
    for i in range(numC - 1):
        if (i == targetC):
            break
        x[:,i] = dataFrame[i]

    for i in range(targetC):
        if dataFrame[targetC][i] == classType[0]: y[i] = 1
        elif dataFrame[targetC][i] == classType[1]: y[i] = 0
        else: 
            print("Error")
            exit(0)


def normalizer_collection(collection, strategies = 'max'):
    collection_clone = collection.select_dtypes(include=[np.number]).astype(float).copy()
    
    if strategies == 'max':
        max_value = np.max(np.abs(collection_clone), axis=0)
        max_value[max_value == 0] = 1
        collection_clone = collection_clone / max_value
    
    # X = - ecarttype/moyen
    elif strategies == 'norme':
        normes = np.linalg.norm(collection_clone, axis=0)
        normes[normes == 0] = 1
        collection_clone = collection_clone / normes
    else:
        raise ValueError("Invalid normalization strategy")

    collection[collection_clone.columns] = collection_clone
    return collection


X_BreastCancer = np.zeros((len(BreastCancer), 31))
Y_BreastCancer = np.zeros(len(BreastCancer))

for i in range(30):
    X_BreastCancer[:,i] = BreastCancer[i]

for i in range(len(BreastCancer)):
    if BreastCancer[1][i] == 'M': Y_BreastCancer[i] = 1
    else: Y_BreastCancer[i] = -0
    
Y_BreastCancer


ValueError: could not convert string to float: 'M'

<font color='red'><b>Question 6:</b></font> Compléter les tableaux comparatifs suivants en repertant les erreurs moyennes sur 20 lancements des modèles de l'Adaline et de la Régression Logistique et pour les trois cas:

 '*' Les vecteurs ne sont pas normalisés
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |




In [22]:
Maxep = 500

def calculate_mean_error():
    pass




print("collection non normalized")



IndentationError: expected an indented block after function definition on line 3 (3916141687.py, line 9)

In [70]:
print("collection normalized with norme")

BreastCancer_normalized_norme = normalizer_collection(BreastCancer, strategies='norme')
Ionosphere_normalized_norme = normalizer_collection(Ionosphere, strategies='norme')
Spambase_normalized_norme = normalizer_collection(Spambase, strategies='norme')

collection normalized with norme


 
 $^n$ Normalisation suivant la stratégie <i>norme</i>
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |

  


In [71]:
print("collection normalized with max")

BreastCancer_normalized_max = normalizer_collection(BreastCancer, strategies='max')
Ionosphere_normalized_max = normalizer_collection(Ionosphere, strategies='max')
Spambase_normalized_max = normalizer_collection(Spambase, strategies='max')

collection normalized with max


 $^m$ Normalisation suivant la stratégie <i>max</i>
    
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |
